## Testiranje modela


In [12]:
from settings import *
from pprint import pformat
from fastai.vision.all import load_learner
import os


model_picks = next(os.walk("./models"))[1]
seen = set()
testees = {}


for model in model_picks:
    spot = Path(garage / model_picks[0])
    try: 
        spot = Path(garage / model)
        print(spot)
    except NameError:
        print(f"model was not defined. Going with {model_picks[0]}.")
    signature = pformat(spot)
    if signature not in seen:
        seen.add(signature)
        try:
            testees[model] = {
                "breath": load_learner(spot/"breath.pkl"),
                "depth": load_learner(spot/"depth.pkl")
            }
        except Exception as ex:
            print(f"Model {model} is not trained yet!")
            print(ex)

testees

models/googlenet_10x
models/alexnet_1x
models/googlenet_1x
models/googlenet_3x
models/alexnet_3x
models/alexnet_10x


{'googlenet_10x': {'breath': <fastai.learner.Learner at 0x7f6ebe64a550>,
  'depth': <fastai.learner.Learner at 0x7f6d8ebf7750>},
 'alexnet_1x': {'breath': <fastai.learner.Learner at 0x7f6d8f3a6550>,
  'depth': <fastai.learner.Learner at 0x7f6d802c6390>},
 'googlenet_1x': {'breath': <fastai.learner.Learner at 0x7f6d9071b110>,
  'depth': <fastai.learner.Learner at 0x7f6d80246290>},
 'googlenet_3x': {'breath': <fastai.learner.Learner at 0x7f6d80259790>,
  'depth': <fastai.learner.Learner at 0x7f6ebc588990>},
 'alexnet_3x': {'breath': <fastai.learner.Learner at 0x7f6d7f592d90>,
  'depth': <fastai.learner.Learner at 0x7f6d7f4a2e10>},
 'alexnet_10x': {'breath': <fastai.learner.Learner at 0x7f6d7f235910>,
  'depth': <fastai.learner.Learner at 0x7f6d7f111e90>}}

### Testiranje na bazi


In [13]:
import pandas as pd

df = pd.read_parquet("data/wikiart_composed.parquet")
df.head()

,artistName,title,year,style,genre,photography,artemis,emotions,Is painting,Face/body,Rating,artistContentId,artistUrl,paintingUrl,localUrl,webUrl,webUrl2
contentId,,,,,,,,,,,,,,,,,
314345,Hans von Aachen,Augustus and the Tiburtine Sibyl,1580,Mannerism (Late Renaissance),mythological painting,0,None,None,NaN,NaN,NaN,314332,hans-von-aachen,augustus-and-the-tiburtine-sibyl-1580,./wikiart/wikidata/images/hans-von-aachen/1580/314345.jpg,https://uploads6.wikiart.org/images/hans-von-aachen/augustus-and-the-tiburtine-sibyl-1580.jpg!Large.jpg,None
314363,Hans von Aachen,Portrait of Jacopo Biliverti,1585,Mannerism (Late Renaissance),portrait,0,None,None,NaN,NaN,NaN,314332,hans-von-aachen,portrait-of-jacopo-biliverti-1585,./wikiart/wikidata/images/hans-von-aachen/1585/314363.jpg,https://uploads2.wikiart.org/images/hans-von-aachen/portrait-of-jacopo-biliverti-1585.jpg!Large.jpg,None
314364,Hans von Aachen,Portrait of Joseph Heintz,1585,Mannerism (Late Renaissance),portrait,0,None,None,NaN,NaN,NaN,314332,hans-von-aachen,portrait-of-joseph-heintz-1585,./wikiart/wikidata/images/hans-von-aachen/1585/314364.jpg,https://uploads8.wikiart.org/images/hans-von-aachen/portrait-of-joseph-heintz-1585.jpg!Large.jpg,None
314371,Hans von Aachen,The rape of Proserpine,1587,Mannerism (Late Renaissance),mythological painting,0,None,None,NaN,NaN,NaN,314332,hans-von-aachen,the-rape-of-proserpine-1587,./wikiart/wikidata/images/hans-von-aachen/1587/314371.jpg,https://uploads5.wikiart.org/images/hans-von-aachen/the-rape-of-proserpine-1587.jpg!Large.jpg,None
314369,Hans von Aachen,The judgment of Paris,1588,Mannerism (Late Renaissance),mythological painting,0,None,None,NaN,NaN,NaN,314332,hans-von-aachen,the-judgment-of-paris-1588,./wikiart/wikidata/images/hans-von-aachen/1588/314369.jpg,https://uploads4.wikiart.org/images/hans-von-aachen/the-judgment-of-paris-1588.jpg!Large.jpg,None


In [14]:
from utils.tester import predict_columns

# prođi kroz bazu u malim batchevima i spremi u datoteke
dataset = df.loc[~df["artemis"].isna() | ~df["emotions"].isna()].sample(1000)
datafiles = {}
for model_name, testee in testees.items(): 
    datafiles[model_name] = predict_columns(testee, dataset, model_name, root);

['artistName', 'title', 'year', 'style', 'googlenet_10x_breath', 'googlenet_10x_breath_probs0', 'googlenet_10x_breath_probs1', 'googlenet_10x_depth', 'googlenet_10x_depth_probs0', 'googlenet_10x_depth_probs1', 'genre', 'artemis', 'emotions', 'webUrl']
Saved the output 1!


In [4]:
# spoji kreirane batch datoteke u jednu
for model_name, addresses in datafiles.items():
    result = pd.DataFrame()
    for adress in addresses:
        data = pd.read_parquet(adress)
        result = pd.concat((result, data))

    result[["artistName", "title", "year", "style", f"{model_name}_breath", f"{model_name}_breath_probs0", f"{model_name}_breath_probs1", f"{model_name}_depth", f"{model_name}_depth_probs0", f"{model_name}_depth_probs1", "genre", "artemis", "emotions", "webUrl"]].to_parquet(f"data/{model_name}/critic_full.parquet")

### Testiranje na web stranicama


In [5]:
from utils.downloader import page_scraper
from utils.tester import test_learners

# "https://aiartists.org/generative-art-design"
# "https://unsplash.com"
# "https://www.metmuseum.org/art/collection/search?showOnly=openAccess"
# "https://www.gettyimages.com"

internet = [
    *page_scraper("https://www.gettyimages.com"),
]

# "https://tse2.mm.bing.net/th?id=OIP.UPZ1-G8gpc5FkNIC2RCWSgHaFj&pid=Api",
# "https://tse2.mm.bing.net/th?id=OIP.if_cidFAKZ49wY7BLA3feQHaGE&pid=Api",
# "https://tse4.mm.bing.net/th?id=OIP.Px4ySbgcqEgFJOhOq8k5mAHaEo&pid=Api",
# "https://tse4.mm.bing.net/th?id=OIP.eNfpYf9Oqyh0u3_b1Eu20wHaGL&pid=Api",
# "https://tse3.mm.bing.net/th?id=OIP.NEkbbYyu56hdqekgPKxmoQAAAA&pid=Api",
# "https://tse3.mm.bing.net/th?id=OIF.jw1Qziy4XGJbDtTYtrBY3Q&pid=Api",
# "https://tse2.mm.bing.net/th?id=OIP.pJwVOqij6rPFrOjcHc1jbAHaKj&pid=Api",
# "https://tse1.mm.bing.net/th?id=OIP.-Hy08hdnnfvCwor0Y_fyGAHaLA&pid=Api",
# "https://tse4.mm.bing.net/th?id=OIP.mUVhO-Zld2qEJKe_r9tJaAHaHn&pid=Api",
# "https://tse2.mm.bing.net/th?id=OIP.DqjuCxnocoFQ0hvIH_Cm8wHaEK&pid=Api",
# "https://tse1.mm.bing.net/th?id=OIP.EFoByYTKu1KbZgoVfocCDAHaFU&pid=Api",
# "https://tse2.mm.bing.net/th?id=OIP.BFRcQqZ5Bpw1a-6u_XoBrwHaFj&pid=Api",
# "https://tse2.mm.bing.net/th?id=OIP.2qqjQQ0wW37Lbvkx50zPxgHaEB&pid=Api"

for model_name, testee in testees.items():
    test_learners(testee, test_set=internet, model_name=model_name, root=root/"artefacts", preview=False)

In [ ]:
import re 

html = 'aaa a ad <img image="tupo-_&$banana.jpeg 300w, milica0087Pilica.png 120w" /> safdsaf saf\
    \safdafa <img asf afasf srcset="data:image/bmp jJAJBDADHHANDKNDKAKNNDDMNANDMND" src="345a"/> \
    saf sa<img data-stretch="false" data-src="https://images.squarespace-cdn.com/content/v1/5c77350965a707ed1710a1bc/1588698548907-RADZYUVD1S7R2R7I0JMM/Generative+Software+Design+Example+-+Procedural+Modeling.jpeg" data-image="https://images.squarespace-cdn.com/content/v1/5c77350965a707ed1710a1bc/1588698548907-RADZYUVD1S7R2R7I0JMM/Generative+Software+Design+Example+-+Procedural+Modeling.jpeg" data-image-dimensions="1500x842" data-image-focal-point="0.5,0.5" alt="Generative Software Design Example - Procedural Modeling.jpeg" data-load="false" src="https://images.squarespace-cdn.com/content/v1/5c77350965a707ed1710a1bc/1588698548907-RADZYUVD1S7R2R7I0JMM/Generative+Software+Design+Example+-+Procedural+Modeling.jpeg" width="1500" height="842" alt="" sizes="100vw" style="display:block;object-fit: cover; width: 100%; height: 100%; object-position: 50% 50%" srcset="https://images.squarespace-cdn.com/content/v1/5c77350965a707ed1710a1bc/1588698548907-RADZYUVD1S7R2R7I0JMM/Generative+Software+Design+Example+-+Procedural+Modeling.jpeg?format=100w 100w, https://images.squarespace-cdn.com/content/v1/5c77350965a707ed1710a1bc/1588698548907-RADZYUVD1S7R2R7I0JMM/Generative+Software+Design+Example+-+Procedural+Modeling.jpeg?format=300w 300w, https://images.squarespace-cdn.com/content/v1/5c77350965a707ed1710a1bc/1588698548907-RADZYUVD1S7R2R7I0JMM/Generative+Software+Design+Example+-+Procedural+Modeling.jpeg?format=500w 500w, https://images.squarespace-cdn.com/content/v1/5c77350965a707ed1710a1bc/1588698548907-RADZYUVD1S7R2R7I0JMM/Generative+Software+Design+Example+-+Procedural+Modeling.jpeg?format=750w 750w, https://images.squarespace-cdn.com/content/v1/5c77350965a707ed1710a1bc/1588698548907-RADZYUVD1S7R2R7I0JMM/Generative+Software+Design+Example+-+Procedural+Modeling.jpeg?format=1000w 1000w, https://images.squarespace-cdn.com/content/v1/5c77350965a707ed1710a1bc/1588698548907-RADZYUVD1S7R2R7I0JMM/Generative+Software+Design+Example+-+Procedural+Modeling.jpeg?format=1500w 1500w, https://images.squarespace-cdn.com/content/v1/5c77350965a707ed1710a1bc/1588698548907-RADZYUVD1S7R2R7I0JMM/Generative+Software+Design+Example+-+Procedural+Modeling.jpeg?format=2500w 2500w" loading="lazy" decoding="async" data-loader="sqs">\
     ffasf asf asf <figure srcset= "https://pili?ćsss3245i.png" =src=><div style="background-image: url(http://vilinkonjic.jpeg )" >saf </div> \
        <img data-stretch="false" data-src="https://images.squarespace-cdn.com/content/v1/5c77350965a707ed1710a1bc/1588703003192-B8US29YF5NDQZQK6WPUN/Silk+Interactive+Generative+Art.jpg" data-image="https://images.squarespace-cdn.com/content/v1/5c77350965a707ed1710a1bc/1588703003192-B8US29YF5NDQZQK6WPUN/Silk+Interactive+Generative+Art.jpg" data-image-dimensions="1958x720" data-image-focal-point="0.5,0.5" alt="Silk Interactive Generative Art.jpg" data-load="false" src="https://images.squarespace-cdn.com/content/v1/5c77350965a707ed1710a1bc/1588703003192-B8US29YF5NDQZQK6WPUN/Silk+Interactive+Generative+Art.jpg" width="1958" height="720" alt="" sizes="100vw" style="display:block;object-fit: cover; width: 100%; height: 100%; object-position: 50% 50%" srcset="https://images.squarespace-cdn.com/content/v1/5c77350965a707ed1710a1bc/1588703003192-B8US29YF5NDQZQK6WPUN/Silk+Interactive+Generative+Art.jpg?format=100w 100w, https://images.squarespace-cdn.com/content/v1/5c77350965a707ed1710a1bc/1588703003192-B8US29YF5NDQZQK6WPUN/Silk+Interactive+Generative+Art.jpg?format=300w 300w, https://images.squarespace-cdn.com/content/v1/5c77350965a707ed1710a1bc/1588703003192-B8US29YF5NDQZQK6WPUN/Silk+Interactive+Generative+Art.jpg?format=500w 500w, https://images.squarespace-cdn.com/content/v1/5c77350965a707ed1710a1bc/1588703003192-B8US29YF5NDQZQK6WPUN/Silk+Interactive+Generative+Art.jpg?format=750w 750w, https://images.squarespace-cdn.com/content/v1/5c77350965a707ed1710a1bc/1588703003192-B8US29YF5NDQZQK6WPUN/Silk+Interactive+Generative+Art.jpg?format=1000w 1000w, https://images.squarespace-cdn.com/content/v1/5c77350965a707ed1710a1bc/1588703003192-B8US29YF5NDQZQK6WPUN/Silk+Interactive+Generative+Art.jpg?format=1500w 1500w, https://images.squarespace-cdn.com/content/v1/5c77350965a707ed1710a1bc/1588703003192-B8US29YF5NDQZQK6WPUN/Silk+Interactive+Generative+Art.jpg?format=2500w 2500w" loading="lazy" decoding="async" data-loader="sqs">'

src = "<(?:img|figure)[\s]*.*?(?:src|image)(?:set)?=[\"'\s]+((?:http)[^'\"\s]+(?:.jpg|.jpeg|.png|.gif|,))[^'\"]*?[\"']+.*?>"
bgurl = "url\([\s]*?((?:http)[^\"'\s]+(?:.jpg|.jpeg|.png))[^\"']*?\)"


re.findall(src, html, re.IGNORECASE), re.findall(bgurl, html, re.IGNORECASE)